## XBRL US API - Python example  
This notebook contains example Python code to use the XBRL US API (https://xbrl.us/home/use/xbrl-api/)    
  
**Made by:** [Ties de Kok](https://www.tiesdekok.com) and [Beth Blankespoor](https://foster.uw.edu/faculty-research/directory/elizabeth-blankespoor/)

## Imports

In [1]:
import os, re, sys, json
import requests

In [2]:
import pandas as pd
import numpy as np

In [3]:
import getpass

## Api documentation

https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails

## Generate an access token
The access token will expire after 60 minutes.

**You have two options:**

**Option 1:** obtain your own credentials

1. Obtain credentials here: https://xbrl.us/home/use/xbrl-api-community/#provisioning

If you use this script on your own computer I recommend using the JSON file as described in option 1a. If you are using Binder I recommend using option 1b.

>**Option 1a:**
>2. Update 'login_cred.json' with your `client_id`, `client_secret`, and `username`  
>3. Set `CREDENTIAL_TYPE` to `LOCAL`
>4. Input your password when asked

>**Option 1b:**
>2. Set `CREDENTIAL_TYPE` to `CLOUD`
>3. Input your details when asked

----

**Option 2:** Use my temporary credentials for demo purposes only

1. Set `CREDENTIAL_TYPE` to `TEMP`
2. Input your email address when asked


In [4]:
CREDENTIAL_TYPE = 'TEMP'

In [5]:
if CREDENTIAL_TYPE == 'TEMP':
    user_email = input(prompt="Please type your email address here: ")
    access_token = requests.get('https://tdekok-xbrlapi.builtwithdark.com/gettoken?platform=aaa-{}'.format(user_email)).text.replace('"', "")
elif CREDENTIAL_TYPE in ['LOCAL', 'CLOUD']:
    endpoint = 'https://api.xbrl.us'
    endpoint_auth = endpoint + '/oauth2/token'
    
    if CREDENTIAL_TYPE == 'LOCAL':
        with open('login_cred.json', 'r') as f:
            login_cred = json.loads(f.read())
            client_id = login_cred['client_id']
            client_secret = login_cred['client_secret']
            username = login_cred['username']
    else:
        client_id = input(prompt='Please input your client id here:')
        client_secret = getpass.getpass(prompt = 'Please input your client secret here:')
        username = input(prompt='Please input your username here:')
        
    password = getpass.getpass(prompt = 'Password: ')
    
    body_auth = {'grant_type' : 'password', 
                'client_id' : client_id, 
                'client_secret' : client_secret, 
                'username' : username,
                'password' : password,
                'platform' : 'uw-ipynb'}
    res = requests.post(endpoint_auth, data=body_auth)
    auth_json = res.json()
    access_token = auth_json['access_token']
else:
    print('Invalid credential type! Use TEMP, LOCAL, or CLOUD. See the instructions above.')

Please type your email address here: your@email.com


## Make a query

### Define the fields you would like returned

In [6]:
fields = ['entity.cik',
         'entity.name.sort(ASC)',
         'dts.id',
         'fact.id',
         'report.filing-date',
         'period.fiscal-year',
         'period.instant',
         'report.document-type',
         'concept.local-name',
         'dimensions.count',
         'dimension.local-name.sort(ASC)',
         'member.local-name',
         'fact.value',
         'unit',
         'fact.decimals',
          'fact.ultimus-index',
         'dimension.namespace',
         'member.namespace',
          'fact.has-dimensions'
         ]

### Define the keywords you'd like to search for

In [7]:
Keyword_List = [
    'intangible'
                ]

### Define the companies you'd like to get extensions for

In [8]:
companies_cik = ['0000789019', ## Microsoft (MSFT)
                 '0001018724', ## Amazon (AMZN)
                 '0001652044', ## Alphabet (GOOG)
                 '0001326801', ## Facebook (FB)
                 '0001065280', ## Netflix (NFLX)
                 '0001166691', ## Comcast (CMCSA)
                 '0001326380', ## GameStop (GME)
                 '0000732717', ## AT&T (T)
                ]

### Define the years you'd like

In [9]:
years = ['2019'] ## Use commas between for multiple years, e.g., '2018','2019'
#years = [str(2013 + i) for i in range(8)] ## Years 2013 to 2020

### Specify if you want dimensions, no dimension, or all values

In [10]:
has_dimensions = 'FALSE'  ## TRUE for require dimensions, FALSE for no dimensions, ALL for all values

### Specify the report types that you want

In [11]:
report_types = ['10-K', '10-K/A']

## Execute query

In [12]:
search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'

In [13]:
params = {'concept.is-base': 'FALSE',
     'period.fiscal-period': 'Y',
     'period.fiscal-year': ','.join(years),
     'unit': 'USD',
     'entity.cik': ','.join(companies_cik),
     'report.document-type': ','.join(report_types),
     }  
if has_dimensions == 'ALL':
    dimension_options = ['TRUE', 'FALSE']
else:
    dimension_options = [has_dimensions]
        
all_res_list = []
for dimensions_param in dimension_options:
    print('Getting the data for: "fact.has-dimensions" = {}'.format(dimensions_param))
    ### Every request will return a max of 2000 results. So we loop until all results are retrieved. 
    done_retrieving_all_results = False
    offset = 0
    while not done_retrieving_all_results:
        params['fact.has-dimensions'] = dimensions_param
        params['fields'] = ','.join(fields) + ',fact.offset({})'.format(offset) 
        res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
        
        ## Interpret as JSON
        res_json = res.json()
            
        ## Get the results
        ### Retrieve the data list
        res_list = res_json['data']
        
        ### Keep only the tags with keyword matches
        result_to_keep = []
        for result in res_list:
            for keyword in Keyword_List:
                if keyword.lower() in result['concept.local-name'].lower():
                    result_to_keep.append(result)
        res_list = result_to_keep
        
        ### Add to the results
        all_res_list += res_list
        
        ## Pagination check
        paging_dict = res_json['paging']
        print('Number of records retrieved: ', paging_dict['count'])
        if paging_dict['count'] >= 2000:
            offset += paging_dict['count']
        else:
            done_retrieving_all_results = True
    
## Convert to a DataFrame
res_df = pd.DataFrame(all_res_list)

print('\nNumber of results that meet the criteria: {}'.format(len(res_df)))

Getting the data for: "fact.has-dimensions" = FALSE
Number of records retrieved:  299

Number of results that meet the criteria: 14


## Show results

In [14]:
columns_to_hide = ['entity.cik', 'fact.id', 'fact.decimals', 'dimension.namespace', 'member.namespace']
columns_to_show = [column for column in res_df.columns if column not in columns_to_hide]

In [15]:
res_df.sort_values(by=['entity.name','dts.id','concept.local-name','fact.ultimus-index','dimension.local-name'], inplace=True)
res_df[columns_to_show].head(20)

,concept.local-name,dimension.local-name,dimensions.count,dts.id,entity.name,fact.has-dimensions,fact.ultimus-index,fact.value,member.local-name,period.fiscal-year,period.instant,report.document-type,report.filing-date,unit
2,DeferredTaxLiabilitiesIndefiniteLivedIntangibles,,0,365998,AT&T INC.,False,1,17355000000,,2019,2020-01-01,10-K,2020-02-20,USD
1,AcquisitionsNetOfCashAcquiredAndPurchasesOfInt...,,0,362758,Alphabet Inc.,False,1,2515000000,,2019,None,10-K,2020-02-04,USD
0,AmortizationAndImpairmentOfIntangibleAssets,,0,362758,Alphabet Inc.,False,1,925000000,,2019,None,10-K,2020-02-04,USD
3,DeferredTaxLiabilitiesPropertyPlantAndEquipmen...,,0,362298,COMCAST CORPORATION,False,1,29387000000,,2019,2020-01-01,10-K,2020-01-30,USD
4,PaymentsToAcquireBusinessesNetOfCashAcquiredAn...,,0,362131,"Facebook, Inc.",False,1,508000000,,2019,None,10-K,2020-01-30,USD
9,IndefiniteandFiniteLivedIntangibleAssetsGross,,0,322837,GameStop Corp.,False,2,133100000,,2019,2019-02-03,10-K,2019-04-02,USD
7,IndefiniteandFiniteLivedIntangibleAssetsNetCar...,,0,322837,GameStop Corp.,False,2,33500000,,2019,2019-02-03,10-K,2019-04-02,USD
6,DeferredTaxAssetsIntangibleAssets,,0,374878,GameStop Corp.,False,1,21800000,,2019,2019-02-03,10-K,2020-03-27,USD
5,IndefiniteandFiniteLivedIntangibleAssetsGross,,0,374878,GameStop Corp.,False,1,133100000,,2019,2019-02-03,10-K,2020-03-27,USD
8,IndefiniteandFiniteLivedIntangibleAssetsNetCar...,,0,374878,GameStop Corp.,False,1,33500000,,2019,2019-02-03,10-K,2020-03-27,USD


## Show dimensions example (if exists)

In [16]:
res_df[res_df['dimensions.count'] > 1].sort_values(by=['entity.name','dts.id','concept.local-name','fact.ultimus-index',
                'dimension.local-name'])[columns_to_show].head(20)

,concept.local-name,dimension.local-name,dimensions.count,dts.id,entity.name,fact.has-dimensions,fact.ultimus-index,fact.value,member.local-name,period.fiscal-year,period.instant,report.document-type,report.filing-date,unit


### Send output to csv

In [17]:
res_df.to_csv('XBRL_Output.csv')     

## If running in Binder, click on the Jupyter icon/name in the upper left corner to see your files, 
## select the file you want, and click Download.